In [19]:
import pandas as pd
import numpy as np
import itertools

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.metrics import mean_squared_error, mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier

from statsmodels.formula.api import ols

from scipy import stats

In [20]:
df_meta = pd.read_csv('/Users/evanjays/Desktop/Programming/Flatiron/CAPSTONE/data_big/ashrae-energy-prediction/building_metadata.csv')
df = pd.read_csv('/Users/evanjays/Desktop/Programming/Flatiron/CAPSTONE/data_big/ashrae-energy-prediction/train.csv')
df_weather = pd.read_csv('/Users/evanjays/Desktop/Programming/Flatiron/CAPSTONE/data_big/ashrae-energy-prediction/weather_train.csv')

In [21]:
meter_data_list = []
df['timestamp'] = pd.to_datetime(df['timestamp'])
df = df[df['meter'] == 0]
for i in df['building_id'].unique():
    df_temp = pd.DataFrame()
    df_temp = df[df['building_id'] == i].copy(deep=True)
    df_temp.set_index('timestamp',inplace=True)
    df_temp = df_temp.groupby('building_id').resample('A').sum()
    df_temp.building_id.ffill(inplace=True)
    meter_data_list.append(df_temp)

df_meter_daily = pd.DataFrame()
for dataframe in meter_data_list:
    dataframe.drop(['building_id'],inplace=True,axis=1)
    dataframe.reset_index(inplace=True) 
    df_meter_daily = pd.concat([df_meter_daily,dataframe])

df_meter_daily.drop(['meter'],axis=1,inplace=True)

In [22]:
df_meter_daily.reset_index(inplace=True,drop=True)

df_meta_energy = df_meta.merge(df_meter_daily,how='right',on='building_id')

In [23]:
df_meta_energy.drop(['building_id','timestamp'],axis=1,inplace=True)

In [24]:
use_dict = {}

for use in X_train['primary_use'].unique():
    if X_train[X_train['primary_use'] == use]['floor_count'].isna().sum() == X_train[X_train['primary_use'] == use]['floor_count'].shape[0]:
        use_dict[use] = 1
    else:
        use_dict[use] = round(X_train[X_train['primary_use'] == use]['floor_count'].median())

def get_num_floors(num_floors):
    if num_floors > 0:
        return num_floors
    else:
        return use_dict[use]

def impute_num_floors(dataframe):
    metadata_list = []
    for idx,use in enumerate(dataframe['primary_use'].unique()):
        df_temp = pd.DataFrame()
        df_temp = dataframe[dataframe['primary_use'] == use].copy(deep=True)
        df_temp['floor_count'] = df_temp['floor_count'].map(get_num_floors)
        metadata_list.append(df_temp.copy(deep=True))

    df_imputed = pd.DataFrame()
    for dataframe in metadata_list:
        df_imputed = pd.concat([df_imputed,dataframe])

    df_imputed.drop(['year_built'],inplace=True,axis=1)
    return df_imputed

df_meta_energy = impute_num_floors(df_meta_energy)

In [25]:
X_train,X_test,y_train,y_test = train_test_split(df_meta_energy.drop(['meter_reading'],axis=1),df_meta_energy['meter_reading'])

In [26]:
X_train

,site_id,primary_use,square_feet,floor_count
284,2,Lodging/residential,32841,1.0
1278,15,Education,98125,1.0
1148,13,Office,35829,1.0
1263,14,Lodging/residential,66794,1.0
1250,14,Education,171084,1.0
...,...,...,...,...
1287,15,Office,14744,1.0
156,2,Office,10135,1.0
1354,15,Lodging/residential,65663,1.0
1307,15,Entertainment/public assembly,111530,1.0


In [28]:
ohe = OneHotEncoder(drop='first')

ohe_train_array = ohe.fit_transform(X_train[['primary_use']])
ohe_train_df = pd.DataFrame(ohe_train_array.todense(),columns=[name[3:] for name in ohe.get_feature_names()])

ohe_test_array = ohe.transform(X_test[['primary_use']])
ohe_test_df = pd.DataFrame(ohe_test_array.todense(),columns=[name[3:] for name in ohe.get_feature_names()])

X_train.reset_index(inplace=True,drop=True)
X_train_ohe = pd.concat([X_train,ohe_train_df],axis=1)
X_train_ohe.drop(['primary_use'],axis=1,inplace=True)

X_test.reset_index(inplace=True,drop=True)
X_test_ohe = pd.concat([X_test,ohe_test_df],axis=1)
X_test_ohe.drop(['primary_use'],axis=1,inplace=True)

In [29]:
ohe_train_array = ohe.fit_transform(X_train_ohe[['site_id']])
ohe_train_df = pd.DataFrame(ohe_train_array.todense(),columns=['site_'+name[3:] for name in ohe.get_feature_names()])

ohe_test_array = ohe.transform(X_test_ohe[['site_id']])
ohe_test_df = pd.DataFrame(ohe_test_array.todense(),columns=['site_'+name[3:] for name in ohe.get_feature_names()])

X_train_ohe.reset_index(inplace=True,drop=True)
X_train_ohe = pd.concat([X_train_ohe,ohe_train_df],axis=1)
X_train_ohe.drop(['site_id'],axis=1,inplace=True)

X_test_ohe.reset_index(inplace=True,drop=True)
X_test_ohe = pd.concat([X_test_ohe,ohe_test_df],axis=1)
X_test_ohe.drop(['site_id'],axis=1,inplace=True)

In [31]:
lr = LinearRegression()

lr.fit(X_train_ohe,y_train)

lr.score(X_test_ohe,y_test)

0.5630506170880045